In [ ]:
 #get all our libraries
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import random
import tensorflow as tf
from scipy.io import loadmat
import scipy.io



In [ ]:
#here we keep the entire history of state sequences and corresponding accuracies
all_acc=[]

In [ ]:
#we initialize our sequence space so that each layer type may appear at each layer number
#its a dictionary so that each tuple can be later indexed in our Q-table
states = {(0, 'i', 1):0,
          (1, 'c1', 1):1,
          (1, 'c2', 1):2,
          (1, 'c3', 1):3,
          (1, 'c4', 1):4,
          (1, 'p1', 1):5,
          (1, 'p2', 1):6,
          (1, 'd1', 0):7,
          (1, 'd2', 0):8,
          (1, 't', 0):9,
          (2, 'c1', 1):10,
          (2, 'c2', 1):11,
          (2, 'c3', 1):12,
          (2, 'c4', 1):13,
          (2, 'p1', 1):14,
          (2, 'p2', 1):15,
          (2, 'd1', 0):16,
          (2, 'd2', 0):17,
          (2, 't', 0):18,
          (3, 'c1', 1):19,
          (3, 'c2', 1):20,
          (3, 'c3', 1):21,
          (3, 'c4', 1):21,
          (3, 'p1', 1):22,
          (3, 'p2', 1):23,
          (3, 'd1', 0):24,
          (3, 'd2', 0):25,
          (3, 't', 0):26,
          (4, 'c1', 1):27,
          (4, 'c2', 1):28,
          (4, 'c3', 1):29,
          (4, 'c4', 1):30,
          (4, 'p1', 1):31,
          (4, 'p2', 1):32,
          (4, 'd1', 0):33,
          (4, 'd2', 0):34,
          (4, 't', 0):35,
          (5, 'c1', 1):36,
          (5, 'c2', 1):37,
          (5, 'c3', 1):38,
          (5, 'c4', 1):39,
          (5, 'p1', 1):40,
          (5, 'p2', 1):41,
          (5, 'd1', 0):42,
          (5, 'd2', 0):43,
          (5, 't', 0):44,
          (6, 'c1', 1):45,
          (6, 'c2', 1):46,
          (6, 'c3', 1):47,
          (6, 'c4', 1):48,
          (6, 'p1', 1):49,
          (6, 'p2', 1):50,
          (6, 'd1', 0):51,
          (6, 'd2', 0):52,
          (6, 't', 0):53,
          (7, 'c1', 1):54,
          (7, 'c2', 1):55,
          (7, 'c3', 1):56,
          (7, 'c4', 1):57,
          (7, 'p1', 1):58,
          (7, 'p2', 1):59,
          (7, 'd1', 0):60,
          (7, 'd2', 0):61,
          (7, 't', 0):62,
          (8, 'c1', 1):63,
          (8, 'c2', 1):64,
          (8, 'c3', 1):65,
          (8, 'c4', 1):66,
          (8, 'p1', 1):67,
          (8, 'p2', 1):68,
          (8, 'd1', 0):69,
          (8, 'd2', 0):70,
          (8, 't', 0):71,
          (9, 't', 0):72
          }
#we initialize all the layer types that we can choose as our actions. They are also indexed for use in Q-table.
#note that input layer type'i' is not included
actions = {'c1':0, 'c2':1, 'c3':2, 'c4':3, 'p1':4, 'p2':5, 'd1':6, 'd2':7, 't':8 }

alpha = 0.01 #Q-learning rate

num_states = len(states) #creates an integer that is the length of all possible states, that we use to initialize the Q-table
num_actions = len(actions) #creates an integer that is the length of all possible actions, that we use to initialize the Q-table

Q_table = np.zeros((num_states, num_actions)) #initialize Q-table
print(Q_table)

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

In [ ]:
#we initialize the Q table to reflect the permissible actions
#non-permissible actions will be zero in the Q-table below
for i in range(len(states)-1):
  if (list (states.keys())[i][2]==0):
    Q_table[i][6]=.5
    Q_table[i][7]=.5
    Q_table[i][8]=.5
  else:
    Q_table[i][0]=.5
    Q_table[i][1]=.5
    Q_table[i][2]=.5
    Q_table[i][3]=.5
    Q_table[i][4]=.5
    Q_table[i][5]=.5
    Q_table[i][6]=.5
    Q_table[i][7]=.5
    Q_table[i][8]=.5
print (Q_table)







[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.  0.  0.  0.  0.  0. 

In [ ]:
import urllib.request
import os

#create a directory for the data
os.makedirs("svhn_dataset", exist_ok=True)

#download SVHN dataset files
urllib.request.urlretrieve("http://ufldl.stanford.edu/housenumbers/train_32x32.mat", "svhn_dataset/train_32x32.mat")
urllib.request.urlretrieve("http://ufldl.stanford.edu/housenumbers/test_32x32.mat", "svhn_dataset/test_32x32.mat")



Files in svhn_dataset directory:
['train_32x32.mat', 'test_32x32.mat']


SVHN Data

In [ ]:

#initialize our desired values for batch size, number of classes, and epochs used in each run through a CNN
batch_size=128
num_classes=10
epochs= 10

im_rows,img_cols,img_chan=32,32,3 #ensure we are using the correct dimensions of the SVHN dataset images

#load training and testing data
train_data = scipy.io.loadmat("svhn_dataset/train_32x32.mat")
test_data = scipy.io.loadmat("svhn_dataset/test_32x32.mat")

#extract images and labels from the loaded data
x_train, y_train = train_data['X'], train_data['y']
x_test, y_test = test_data['X'], test_data['y']

#transpose the image data to have the correct dimensions
x_train = x_train.transpose((3, 0, 1, 2))
x_test = x_test.transpose((3, 0, 1, 2))

#turn it into vector
y_train = y_train.flatten()
y_test = y_test.flatten()

#Make the 10 value be a 0 as it
y_train = [0 if label == 10 else label for label in y_train]
y_test = [0 if label == 10 else label for label in y_test]

y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test, num_classes)

x_train, x_test = x_train / 255.0, x_test / 255.0

# Print shapes of the datasets
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)



x_train shape: (73257, 32, 32, 3)
y_train shape: (73257, 10)
x_test shape: (26032, 32, 32, 3)
y_test shape: (26032, 10)


In [ ]:
#this function takes in the previous state and action it returns the new state chosen
#it uses the indicators to ensure that once the network has reached a dense or terminal layer, -
#it cannot return to a convolutional or maxpooling layer
def transition(state, action):
  layer_num, layer_type, indicator = state
  layer_num += 1
  if action in ['d1','d2' ,'t']:
    indicator = 0
  else:
    indicator = 1
  new_state = (layer_num, action, indicator)
  return new_state

#this function returns the permissible actions at a given state
#the indicator state variable indicates whether it is possible for the next layer to be convolutional or maxpooling
#1 indicates that all layers types are still permissibile
#0 indicates that only dense layers or the termination layer are allowed
#the first if statements ensures that there is a max number of 8 layers allowed
def permissible_actions(state):
  layer_num, layer_type, indicator = state
  if layer_num==8:
    actions=['t']
  elif layer_num == 5:
    actions= ['d1','d2','t']
  else:
    if indicator==0:
      actions = ['d1', 'd2', 't']
    else:
      actions = ['c1', 'c2','c3', 'c4', 'p1','p2', 'd1','d2', 't']

  return actions

#this function implements a part of the Q-learning algorithm that samples a new network architecture based on the current Q-table
#it starts from an initial state and iteratively selects actions until a terminal action is chosen, forming a path through the state-action space.
#the selection of actions is based on an epsilon-greedy strategy, which balances exploration (random action selection) -
#-and exploitation (choosing the action with the highest expected reward, based on the Q-table)
def sample_new_network(epsilon, Q_table):
  state_seqn = [(0, 'i',1)]
  action_seqn = []

  while (action_seqn==[] or action_seqn[-1]!='t'):
    beta = random.random()
    if beta > epsilon:
      state = (states.get(state_seqn[-1]))
      permissible_indices = []
      for i in permissible_actions(state_seqn[-1]):
        permissible_indices.append(actions.get(i))
      q_values = list(Q_table[state])
      permissible_q_values = [q_values[i] for i in permissible_indices]
      max_action = max(permissible_q_values)
      max_action_index = permissible_q_values.index(max_action)+(len(q_values)-len(permissible_q_values))
      u = list(actions.keys())[max_action_index]
      s = transition(state_seqn[-1], u)
    else:
      u = random.choice(permissible_actions(state_seqn[-1]))
      s = transition(state_seqn[-1], u)
    action_seqn.append(u)
    if u!='t':
      state_seqn.append(s)

  return state_seqn, action_seqn
#this function updates Q-values based on the received accuracy of the sampled network
#it iterate backwards through the state-action sequence to update the Q-values for all state-action pairs-
#-which allows the algorithm to propagate the accuracy information back to the initial states.
def update_q_values(Q, state_seqn, action_seqn, accuracy):
    Q[states.get(state_seqn[-1])][actions.get(action_seqn[-1])] = (1 - alpha) * Q[states.get(state_seqn[-1])][actions.get(action_seqn[-1])] + alpha * accuracy
    for i in range(len(state_seqn) - 2, -1, -1):
        best_future_q = max(Q[states.get(state_seqn[i])][actions.get(future_action)] for future_action in permissible_actions(state_seqn[i+1]))
        Q[states.get(state_seqn[i])][actions.get(action_seqn[i])] = (1 - alpha) * Q[states.get(state_seqn[i])][actions.get(action_seqn[i])] + alpha * best_future_q




In [ ]:
import gc #imports a garbage collector that let us delete the cache

In [ ]:
#this function creates the actual convolution network by adding the layers given by the sequence of chosen states by the agent
#the first double-if statement is in place for time efficiency, so that if a state sequence has already appeared in the-
#-model we just return the accuracy of the episode instead of rerunning that CNN
#this function is where we choose what exact layers we want the agent to choose from
#finally, it compiles the given CNN, and compares the output to the true categories so as to obtain an accuracy score
def train(state_seqn):
  if len(all_acc)>0:
    first_elements = [x[0] for x in all_acc]
    if state_seqn in first_elements:
      return all_acc[first_elements.index(state_seqn)][1]

  model = Sequential()
  model.add(keras.Input(shape=(32,32,3)))
  if len(state_seqn)==1:
    model.add(Flatten())
    model.add(Dense(10, activation='Softmax', input_shape=(32, 32, 3)))
  else:
      counter =0

      for state in state_seqn[1:]:
        if state[1] == 'c1':
          model.add(Conv2D(32, kernel_size= (2, 2), padding='same',activation='relu'))
        elif state[1] == 'c2':
          model.add(Conv2D(64, kernel_size= (2, 2), padding='same',activation='relu'))
        elif state[1] == 'c3':
          model.add(Conv2D(32, kernel_size= (3, 3), padding='same',activation='relu'))
        elif state[1] == 'c4':
          model.add(Conv2D(64, kernel_size= (3, 3), padding='same',activation='relu'))
        elif state[1] == 'p1':
          model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
        elif state[1] == 'p2':
          model.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
        elif state[1] == 'd1':
          model.add(Flatten())
          model.add(Dense(128, activation='relu'))
        elif state[1] == 'd2':
          model.add(Flatten())
          model.add(Dense(64, activation='relu'))
        if (counter%2 == 0):
          model.add(Dropout(0.2))
        counter+=1
      model.add(Flatten())
      model.add(Dense(10, activation='softmax'))

  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(learning_rate = 1e-2),
              metrics=['accuracy'])


  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=0,
            validation_data=(x_test, y_test))
  score = model.evaluate(x_test, y_test, verbose=0)

  del model
  gc.collect()
  keras.backend.clear_session()
  return score[1]




In [ ]:
#this is the main function of our Q-learning model that uses all of the functions created above
#the first for loop runs the episodes where for each a new neural network architecture is sampled-
#deciding the architecture based on the current Q-table and the exploration rate
#the second for loop implements the use of a replay memory for updating Q-values multiple times-
#-within each episode which allows for improved stability and efficiency of reinforcement learning algorithms

def meta_qnn_algorithm(epsilon, m=10, k=7):
    """MetaQNN main algorithm."""
    SS_acc_list=[()]
    replay_memory = []


    for episode in range(1, m + 1):
        state_seqn, action_seqn = sample_new_network(epsilon, Q_table)
        accuracy = train(state_seqn)

        tup=(state_seqn,accuracy)
        SS_acc_list=[]
        SS_acc_list.append(tup)
        replay_memory.append((state_seqn, action_seqn, accuracy))
        all_acc.append(tup)

        for memory in range(1, k+1):
            Ssample, Usample, ACCsample = random.choice(replay_memory)
            update_q_values(Q_table, Ssample, Usample, ACCsample)
    print(epsilon)

In [ ]:
#finally we run the meta_qnn_algorithm multiple times with various epsilon values
#we start with a large number at epsilon=1 to give it appropriate time to explore-
#-before it eventually chooses more 'greedy' options
#we decrement epsilon by .1 making the agent more 'greedy' until it reaches 0.1-
#-where it should converge to an optimal CNN
for i in range(8):
  meta_qnn_algorithm(1)
for i in range(2):
  meta_qnn_algorithm(0.9)
for i in range(2):
  meta_qnn_algorithm(0.8)
for i in range(2):
  meta_qnn_algorithm(0.7)
for i in range(2):
  meta_qnn_algorithm(0.6)
for i in range(2):
  meta_qnn_algorithm(0.5)
for i in range(2):
  meta_qnn_algorithm(0.4)
for i in range(2):
  meta_qnn_algorithm(0.3)
for i in range(3):
  meta_qnn_algorithm(0.2)
for i in range(3):
  meta_qnn_algorithm(0.1)

Epoch 1/10
573/573 [==============================] - 7s 6ms/step - loss: 2.2510 - accuracy: 0.1824 - val_loss: 2.2323 - val_accuracy: 0.1959
Epoch 2/10
573/573 [==============================] - 3s 5ms/step - loss: 2.2315 - accuracy: 0.1977 - val_loss: 2.2017 - val_accuracy: 0.2217
Epoch 3/10
573/573 [==============================] - 3s 5ms/step - loss: 2.1981 - accuracy: 0.2259 - val_loss: 2.1566 - val_accuracy: 0.2533
Epoch 4/10
573/573 [==============================] - 3s 5ms/step - loss: 2.1464 - accuracy: 0.2599 - val_loss: 2.0899 - val_accuracy: 0.3016
Epoch 5/10
573/573 [==============================] - 3s 5ms/step - loss: 2.0764 - accuracy: 0.2949 - val_loss: 2.0127 - val_accuracy: 0.3262
Epoch 6/10
573/573 [==============================] - 3s 5ms/step - loss: 1.9933 - accuracy: 0.3294 - val_loss: 1.9167 - val_accuracy: 0.3924
Epoch 7/10
573/573 [==============================] - 3s 5ms/step - loss: 1.9058 - accuracy: 0.3618 - val_loss: 1.8288 - val_accuracy: 0.4077
Epoch 

In [ ]:
print(all_acc) #we print out for the sake of graphing and observing the accuracies of different CNN structures

[([(0, 'i', 1), (1, 'c3', 1), (2, 'p1', 1), (3, 'd2', 0)], 0.47157344222068787), ([(0, 'i', 1), (1, 'c1', 1), (2, 'd2', 0), (3, 'd1', 0)], 0.6152427792549133), ([(0, 'i', 1), (1, 'c1', 1), (2, 'c2', 1), (3, 'd1', 0), (4, 'd1', 0)], 0.6437845826148987), ([(0, 'i', 1), (1, 'd2', 0), (2, 'd2', 0), (3, 'd1', 0)], 0.42609095573425293), ([(0, 'i', 1), (1, 'c1', 1), (2, 'c4', 1)], 0.4880915880203247), ([(0, 'i', 1), (1, 'c1', 1), (2, 'd1', 0)], 0.6417102217674255), ([(0, 'i', 1), (1, 'c1', 1), (2, 'c3', 1), (3, 'c3', 1), (4, 'd1', 0)], 0.7429701685905457), ([(0, 'i', 1)], 0.21074062585830688)]


In [ ]:
print(Q_table) #we print out the final Q-table for observation



[[0.12486917 0.12238454 0.14077126 0.18298549 0.1130643  0.11265022
  0.14694526 0.130832   0.33637777]
 [0.1        0.1        0.1        0.1        0.1        0.1
  0.1        0.1        0.1       ]
 [0.1        0.1        0.1        0.1        0.1        0.10012176
  0.10012176 0.1        0.12672317]
 [0.1        0.1        0.1        0.1        0.1        0.1
  0.1        0.1        0.1       ]
 [0.1        0.1        0.1        0.1        0.1        0.1
  0.1        0.1        0.12436632]
 [0.1        0.1        0.10038949 0.1        0.1        0.1
  0.1        0.1        0.11391241]
 [0.1        0.1        0.1        0.1        0.1        0.1
  0.1        0.1        0.1       ]
 [0.         0.         0.         0.         0.         0.
  0.10075488 0.10460602 0.17340449]
 [0.         0.         0.         0.         0.         0.
  0.10280151 0.10010611 0.15771688]
 [0.         0.         0.         0.         0.         0.
  0.1        0.1        0.1       ]
 [0.1        0.1   

In [ ]:
'''
import csv
import pickle
import ast
import json


csv_file_path = 'table_failed.csv'

np.savetxt(csv_file_path, Q_table, delimiter=",")
# Now, let's import the CSV file back into another NumPy array
Q_values_new = np.genfromtxt(csv_file_path, delimiter=",")

# Display the imported array
print(Q_values_new)
'''